In [79]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
import re
from datetime import datetime, timezone,timedelta
from dateutil import parser
from nba_api.stats.endpoints import scoreboardv2, ScoreboardV2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore, BoxScore
nba_teams = teams.get_teams()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [80]:
yesterday = datetime.today() - timedelta(days=1) 
yesterday = yesterday.strftime("%Y-%m-%d")


In [81]:
lst = []
szn_25_26 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2025-10-21",
                                      date_to_nullable=yesterday,
                                      season_type_nullable="Regular Season",
                                      league_id_nullable="00")
szn_25_26 = szn_25_26.get_data_frames()[0]


In [82]:
szn_25_26 = pd.DataFrame(szn_25_26)
ids = list(szn_25_26['TEAM_ID'].unique())
id_to_name  = {team['id']: team['full_name'] for team in nba_teams}

In [83]:
szn_25_26["TotalFGMade"] = szn_25_26.groupby('GAME_ID')['FGM'].transform('sum')
szn_25_26["TotalFGAtt"] = szn_25_26.groupby('GAME_ID')['FGA'].transform('sum')
szn_25_26['Total3PM'] = szn_25_26.groupby('GAME_ID')['FG3M'].transform('sum')
szn_25_26["TotalTOV"] = szn_25_26.groupby('GAME_ID')['TOV'].transform('sum')
szn_25_26["TotalFTA"] = szn_25_26.groupby('GAME_ID')['FTA'].transform('sum')
szn_25_26['TotalOREB'] = szn_25_26.groupby('GAME_ID')['OREB'].transform('sum')
szn_25_26['TotalGamePoints'] = szn_25_26.groupby('GAME_ID')['PTS'].transform('sum')
szn_25_26['OppOREB'] = szn_25_26['TotalOREB'] - szn_25_26['OREB']
szn_25_26['TDREB'] = szn_25_26.groupby('GAME_ID')['DREB'].transform('sum')
szn_25_26['TotalGameFTM'] = szn_25_26.groupby('GAME_ID')['FTM'].transform('sum')
szn_25_26['TotalDREB'] = szn_25_26.groupby('GAME_ID')['DREB'].transform('sum')
szn_25_26['TotalAST'] = szn_25_26.groupby('GAME_ID')['AST'].transform('sum')
szn_25_26['TotalSTL'] = szn_25_26.groupby('GAME_ID')['STL'].transform('sum')
szn_25_26['TotalBLK'] = szn_25_26.groupby('GAME_ID')['BLK'].transform('sum')
szn_25_26['TotalPF'] = szn_25_26.groupby('GAME_ID')['PF'].transform('sum')
szn_25_26['TotalTOV'] = szn_25_26.groupby('GAME_ID')['TOV'].transform('sum')

In [84]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 4)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return round(float(x['OREB']) / (float(x['OREB'] + x['OPPDREB'])),4)
def FTRATE(x):
    return round(float(x['FTA']) / float(x['FGA']),4)
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 4)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 4)
def DREBP(x):
    return round(float(x['DREB']) / float(x['DREB'] + x['OppOREB']),4)
def OppFTRATE(x):
    return round(float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA']),4)
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)


In [85]:
szn_25_26['EFG'] = szn_25_26.apply(EFG, axis=1)
szn_25_26['TOVP'] = szn_25_26.apply(TOVP, axis=1)
szn_25_26['OREBP'] = szn_25_26.apply(OREBP, axis=1)
szn_25_26['FTRATE'] = szn_25_26.apply(FTRATE, axis=1)
szn_25_26['OppEFG'] = szn_25_26.apply(OppEFG, axis=1)
szn_25_26['OppTOVP'] = szn_25_26.apply(OppTOVP, axis=1)
szn_25_26['DREBP'] = szn_25_26.apply(DREBP, axis=1)
szn_25_26['OppFTRATE'] = szn_25_26.apply(OppFTRATE, axis=1)
szn_25_26["AWAY"] = szn_25_26.apply(away, axis=1)
szn_25_26["HOME"] = szn_25_26.apply(home, axis=1)
szn_25_26['TEAMPIE'] = szn_25_26.apply(teamPIE, axis=1)

In [86]:
szn_25_26.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [87]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined

In [88]:
szn_25_26['_original_index'] = range(len(szn_25_26))
x_sorted = szn_25_26.sort_values(['TEAM_ID', 'SEASON_ID', 'GAME_DATE']).reset_index(drop=True)
ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE']
for col in ewma_cols:
    x_sorted[f"EWMA {col}"] = (
    x_sorted.groupby(['TEAM_ID', 'SEASON_ID'])[col].apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean().round(4)).reset_index(level=[0, 1], drop=True))
x_sorted = x_sorted.sort_values('_original_index').reset_index(drop=True)
x_sorted = x_sorted.drop(columns=['_original_index'])

In [89]:
query = "SELECT * FROM [2024-2025 Team Data Combined] where [game_date] = '2025-04-13'"
z = pd.read_sql_query(query, conn)

In [90]:
elo_dct = {}
for index, rows in z.iterrows():
    elo_dct[rows['home_TEAM_ID']] = rows['home_ELO']
    elo_dct[rows['away_TEAM_ID']] = rows['away_ELO']
elo_dct.update((x, 1500 + ((y-1500) * (1 - (1/3)))) for x, y in elo_dct.items())
def ELO(x):
    prob_home = 1 / (1 + 10**((team_id_dict[x['away_TEAM_ID']] - team_id_dict[x['home_TEAM_ID']] - 100)/400))
    prob_away = 1 - prob_home
    if (x['home_win'] == 1):
        pre_elo_home = team_id_dict[x['home_TEAM_ID']]
        team_id_dict[x['home_TEAM_ID']] = pre_elo_home + 20 * (1 - prob_home)
        pre_elo_away = team_id_dict[x['away_TEAM_ID']]
        team_id_dict[x['away_TEAM_ID']] = pre_elo_away + 20 * (0 - prob_away)
    else:
        pre_elo_home = team_id_dict[x['home_TEAM_ID']]
        team_id_dict[x['home_TEAM_ID']] = pre_elo_home + 20 * (0 - prob_home)
        pre_elo_away = team_id_dict[x['away_TEAM_ID']]
        team_id_dict[x['away_TEAM_ID']] = pre_elo_away + 20 * (1 - prob_away)
    return pre_elo_home, pre_elo_away

In [91]:
x_sorted = combine_game_rows(x_sorted)
x_sorted = x_sorted.sort_values(by='GAME_DATE', ascending=True)
x_sorted[['home_ELO', 'away_ELO']] = x_sorted.apply(ELO, axis=1, result_type='expand')
%store team_id_dict

Stored 'team_id_dict' (dict)


/Users/rohilverma/Library/Python/3.9/lib/python/site-packages/IPython/extensions/storemagic.py:229: UserWarning: using autorestore/team_id_dict requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [92]:
x_sorted.drop(columns=['AWAY_y','HOME_y', 'away_is_home','home_is_home','AWAY_x', 'HOME_x',], inplace=True)

In [93]:
x_sorted.to_sql("2025-2026 Team Data Combined", conn, if_exists="replace", index=False)

173

In [94]:
file = 'nba.db'
table = "[2025-2026 Team Data Combined]"
query = f"SELECT * FROM {table}"
x = pd.read_sql_query(query, conn)

In [95]:
x

,GAME_ID,GAME_DATE,SEASON_ID,home_TEAM_ID,home_TEAM,home_TEAM_NAME,MATCHUP_x,home_WL,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_TotalGamePoints,home_TotalGameFTM,home_TotalDREB,home_TotalAST,home_TotalSTL,home_TotalBLK,home_TotalPF,home_EFG,home_TOVP,home_OREBP,home_FTRATE,home_OppEFG,home_OppTOVP,home_DREBP,home_OppFTRATE,home_TEAMPIE,home_EWMA PTS,home_EWMA FGA,home_EWMA FG_PCT,home_EWMA FG3M,home_EWMA FG3A,home_EWMA FG3_PCT,home_EWMA FTM,home_EWMA FTA,home_EWMA FT_PCT,home_EWMA OREB,home_EWMA DREB,home_EWMA AST,home_EWMA STL,home_EWMA TOV,home_EWMA PF,home_EWMA PLUS_MINUS,home_EWMA EFG,home_EWMA TOVP,home_EWMA OREBP,home_EWMA FTRATE,home_EWMA OppEFG,home_EWMA OppTOVP,home_EWMA DREBP,home_EWMA OppFTRATE,home_EWMA TEAMPIE,away_TEAM_ID,away_TEAM,away_TEAM_NAME,MATCHUP_y,away_WL,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_TotalGamePoints,away_TotalGameFTM,away_TotalDREB,away_TotalAST,away_TotalSTL,away_TotalBLK,away_TotalPF,away_EFG,away_TOVP,away_OREBP,away_FTRATE,away_OppEFG,away_OppTOVP,away_DREBP,away_OppFTRATE,away_TEAMPIE,away_EWMA PTS,away_EWMA FGA,away_EWMA FG_PCT,away_EWMA FG3M,away_EWMA FG3A,away_EWMA FG3_PCT,away_EWMA FTM,away_EWMA FTA,away_EWMA FT_PCT,away_EWMA OREB,away_EWMA DREB,away_EWMA AST,away_EWMA STL,away_EWMA TOV,away_EWMA PF,away_EWMA PLUS_MINUS,away_EWMA EFG,away_EWMA TOVP,away_EWMA OREBP,away_EWMA FTRATE,away_EWMA OppEFG,away_EWMA OppTOVP,away_EWMA DREBP,away_EWMA OppFTRATE,away_EWMA TEAMPIE,home_win,home_ELO,away_ELO
0,0022500002,2025-10-21,22025,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,240,109,42,77,0.545,8,32,0.250,17,28,0.607,7,32,39,23,7,2,19,21,-10.0,228,43,63,52,17,6,48,0.5974,0.1754,0.1842,0.3636,0.5962,0.1655,0.7805,0.3718,0.4543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612744,GSW,Golden State Warriors,GSW @ LAL,W,241,119,38,78,0.487,17,40,0.425,26,29,0.897,9,31,40,29,10,4,18,27,10.0,228,43,63,52,17,6,48,0.5962,0.1655,0.2195,0.3718,0.5974,0.1754,0.8158,0.3636,0.5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1603.004899,1534.636288
1,0022500001,2025-10-21,22025,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,290,125,46,104,0.442,13,52,0.250,20,25,0.800,11,27,38,29,12,4,11,27,1.0,249,47,63,52,18,9,53,0.5048,0.0873,0.2340,0.2404,0.5000,0.1659,0.6279,0.3196,0.5155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612745,HOU,Houston Rockets,HOU @ OKC,L,292,124,43,97,0.443,11,39,0.282,27,31,0.871,16,36,52,23,6,5,22,26,-1.0,249,47,63,52,18,9,53,0.5000,0.1659,0.3721,0.3196,0.5048,0.0873,0.7660,0.2404,0.4845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1698.989044,1597.568168
2,0022500084,2025-10-22,22025,1610612741,CHI,Chicago Bulls,CHI vs. DET,W,242,115,39,87,0.448,11,30,0.367,26,32,0.813,12,38,50,29,6,8,17,23,4.0,226,44,71,58,14,18,52,0.5115,0.1440,0.2667,0.3678,0.4794,0.0844,0.7451,0.2680,0.5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612765,DET,Detroit Pistons,DET @ CHI,L,239,111,43,97,0.443,7,24,0.292,18,26,0.692,13,33,46,29,8,10,10,29,-4.0,226,44,71,58,14,18,52,0.4794,0.0844,0.2549,0.2680,0.5115,0.1440,0.7333,0.3678,0.4680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1520.911034,1632.612163
3,0022500087,2025-10-22,22025,1610612762,UTA,Utah Jazz,UTA vs. LAC,W,240,129,48,87,0.552,17,44,0.386,16,20,0.800,13,33,46,38,11,6,12,21,21.0,237,32,56,66,19,12,42,0.6494,0.1113,0.3611,0.2299,0.5227,0.1336,0.6875,0.2386,0.6228,NaN,NaN,Na